In [5]:
import os
import json
import gc

#import torch 
#import torch.nn.functional as F
#from transformers import AutoModelForCausalLM,  StoppingCriteria, StoppingCriteriaList, AutoTokenizer,BitsAndBytesConfig, pipeline, AutoConfig , TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import AutoTokenizer
#import trl
from trl import apply_chat_template
#from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
#from peft import LoraConfig, PeftConfig
from openai import AzureOpenAI


from tqdm import tqdm_notebook
from datasets import Dataset

#import bitsandbytes as bnb

KeyboardInterrupt: 

In [ ]:
TRAIN_DATASET_PATH = 'datasets/train.json'
TEST_DATASET_PATH = 'datasets/test.json'
DEV_DATASET_PATH = 'datasets/dev.json'
model_id = "microsoft/Phi-3-mini-128k-instruct"

endpoint = os.getenv("ENDPOINT_URL", "")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "")  

# Initialize Azure OpenAI client with key-based authentication
oai_client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",  
) 


reason_system_prompt = '''
You are a legal agent. You are provided with statements from a contract and a hypothesis and an annotation json.
Definition of Input is as follows:
1. Contract: A legal NDA provided by the user in line by line format as described below
2. Hypothesis: A statement that user asks corresponding to the contract
3. Annotation: A json object containing two keys, choice and spans. 

Choice contains following values:

3.1. Entailment: The hypothesis is directly supported by one or more of the statements from the contract, meaning if the statement is true, then the hypothesis is true. The implication should be unambiguous and without assumptions.
3.2. Contradiction: The hypothesis contradicts one or more of the statements from the contract, meaning if the statement is true, then the hypothesis is false. This implication should be unambiguous and without assumptions.
3.3. Not Mentioned: In cases where none of the above conclusions can be drawn without assumptions.

Spans: Statement ids which supports the choice

Your task is to evalaute and tell your thoughts for the choice given in the annotation and why the given statement ids are selected. 
Dont repeat statement from contract, instead refer to them by id value e.g "statement_id_1 mentions this". 
Give your thought step wise:
1. First refer to each span mentioned and tell why do you think its choosen.
2. Then give your opinion on how the statement supports the choice.
3. If you dont agree with the choice mentioned, return does_not_agree yes else no.




Input Format:
Contract:
[[statement_id_1]]: <1st statement>
[[statement_id_2]]: <2nd statement>
..

Hypothesis:
<hypothesis_statement>

Annotation:
```json
{
    "choice": "<one of Entailment, Contradiction, NotMentioned>",
    "spans": ["<statement ids that supports the above choice>"],
}
```

Output Format:
```json
{
    "thoughts": "<thoughts for annotation in less than 150 words>",
    "does_not_agree": "<yes/no>"
}
```
'''

reason_user_prompt = '''
Contract: 
{0}

Hypothesis:
{1}

Annotation:
{2}
'''

ft_system_prompt = '''
You are a legal agent. You are provided with statements from a contract and a hypothesis. Your task is to evaluate whether the hypothesis statement is:
1. Entailment: The hypothesis is directly supported by one or more of the statements from the contract, meaning if the statement is true, then the hypothesis is true. The implication should be unambiguous and without assumptions.
2. Contradiction: The hypothesis contradicts one or more of the statements from the contract, meaning if the statement is true, then the hypothesis is false. This implication should be unambiguous and without assumptions.
3. Not Mentioned: In cases where none of the above conclusions can be drawn without assumptions.


Input Format:
Contract:
[[statement_id_1]]: <1st statement>
[[statement_id_2]]: <2nd statement>
..

Hypothesis:
<hypothesis_statement>

Output Format:
```json
{
    "thoughts": "<explain the thought behind selection>"
    "choice": "<one of Entailment, Contradiction, NotMentioned>",
    "spans": ["<mention most relevant statement ids that directly support the choice in case of Entailement and Contradictory, otherwise keep empty>"],
}
```
'''

ft_user_prompt = '''
Contract:
{0}
Hypothesis:
{1}
'''


chat_template = '''
<|system|>
{0}
<|user|>
{1}<|end|>
<|assistant|>
'''

response_template = '''<|assistant|>'''

In [ ]:
def load_dataset(path):
    with open(path,'r') as f:
        dataset =  json.load(f)
    return dataset

def get_embedding(model, tokenizer, text: str, aggregation_method: str = None):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    last_hidden_state = outputs.hidden_states[-1]

    if aggregation_method == 'mean':
        # Mean pool the token embeddings to get a sentence embedding
        embedding = last_hidden_state.mean(dim=1)
    elif aggregation_method == 'last_token':
        # Get the hidden state of the last token
        embedding = last_hidden_state[:, -1, :]  # Shape: [batch_size, hidden_size]
    else:
        # If no valid aggregation method is specified, return the last hidden state
        embedding = last_hidden_state

    # Normalize the embedding (if applicable)
    if aggregation_method in ['mean', 'last_token']:
        embedding = F.normalize(embedding, p=2, dim=1)

    return embedding

def construct_statement_annotation_pair(dataset_input):
    statement_annotation_pair = {}
    for i,document in enumerate(dataset_input):
        #print(document['id'])
        statement_annotation_pair[i] = {}
        statement_annotation_pair[i]['statement'] = ''
        for idx,span in enumerate(document['spans']):
            #print(span)
            statement_annotation_pair[i]['statement'] += f'''[[statement_id_{idx}]]: {document['text'][span[0]:span[1]]}\n'''
            statement_annotation_pair[i]['target'] =  document['annotation_sets'][0]
            statement_annotation_pair[i]['doc_id'] = document['id']
    return statement_annotation_pair

def get_reason_from_gpt4(contract,hypothesis,annotation):  
    # Prepare the chat prompt  
    chat_prompt = [
    {
        "role": "system",
        "content": reason_system_prompt
    },
        {
            "role": "user",
            "content": reason_user_prompt.format(contract,hypothesis,annotation)
        }
    ]   

    # Generate the completion  
    completion = oai_client.chat.completions.create(  
        model=deployment,  
        messages=chat_prompt,  
        max_tokens=200,  
        temperature=0.7,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,  
        stop=None,  
        stream=False,
        response_format = { "type": "json_object" }
    )  
    response = json.loads(json.loads(completion.to_json())["choices"][0]['message']['content'])
    return response['thoughts'],response['does_not_agree']



def prepare_dataset_for_finetuning_method1(train_contract_statements_and_annotations,hypothesis_set,tokeniser,max_tokens=8192,augment_reason=True,test_data=False):
    '''
    {
        "prompt": [{"role": "user", "content": "What color is the sky?"}],
        "completion": [{"role": "assistant", "content": "It is blue."}]
    }
    '''
    dataset = []
    total_points = 0
    skipped_points = 0
    for idx,contract in tqdm_notebook(train_contract_statements_and_annotations.items()):
        #current_system_prompt = ft_system_prompt.format(contract['statement'])
        for key,value in tqdm_notebook(hypothesis_set.items()):
            #current_user_prompt = f"Evaluate following hypothesis: \n{value['hypothesis']}"
            response = "","no"
            if augment_reason:
                try:
                    response=get_reason_from_gpt4(contract['statement'],value['hypothesis'],contract['target']['annotations'][key])
                except Exception as e:
                    print(e)
                #print(response[0],response[1])
                contract['target']['annotations'][key] = { **{"thoughts":response[0]}, **contract['target']['annotations'][key]}
                #print(contract['target']['annotations'][key])                                 
            #print(contract['target']['annotations'][key])
            data_point = {
                "doc_id": idx,
                "prompt": [
                    {
                        "role": "system",
                        "content": ft_system_prompt
                    },
                    {
                        "role": "user",
                        "content": ft_user_prompt.format(contract['statement'],value['hypothesis'])
                    }
                ],
                "completion": [
                    {
                        "role": "assistant",
                        "content": f"```json\n{json.dumps(contract['target']['annotations'][key])}\n```"
                    }
                ]
            }
            chatified_text = apply_chat_template(data_point,tokeniser)
            chatified_text['is_contradicted_by_gpt4'] = response[1]
            chatified_text['doc_id'] = contract['doc_id']
            prompt_tokens = len(tokeniser(chatified_text['prompt'],truncation=False)['input_ids'])
            completion_tokens = len(tokeniser(chatified_text['completion'],truncation=False)['input_ids'])
            total_points += 1
            if(prompt_tokens  + completion_tokens <= max_tokens):
#                 print(chatified_text)
                dataset.append(chatified_text)
            else:
                skipped_points += 1

    return Dataset.from_list(dataset),total_points,skipped_points
    
    

'''
With All Statements
'''
def generate_message_target_pair(idx,train_contract_statements_and_annotations,nda,model=None,tokeniser=None):
    sim_scores = {}
    example_data = train_contract_statements_and_annotations[idx]
    current_system_prompt = system_prompt.format(example_data['statement'])
    hypothesis_embed = get_embedding(model,tokeniser,hypothesis_set[nda]['hypothesis'],'mean')
    if model is not None and tokeniser is not None:
        for idx,statement in enumerate(example_data['statement'].split('\n')):
            if len(statement) > 0:
                statement_embed = get_embedding(model,tokeniser,statement,'mean')
                sim_scores[statement.split(']]')[0].split('[[')[-1]] = ((statement_embed@hypothesis_embed.T).item())

    current_user_prompt = hypothesis_set[nda]['hypothesis']
    chat_template_current = chat_template.format(current_system_prompt,current_user_prompt)
    messages = [ 
        {"role": "system", "content": current_system_prompt}, 
        {"role": "user", "content": current_user_prompt},
    ]
    target = example_data['target']['annotations'][nda]

    # Get the top 5 items sorted by value
    top_5_statements = sorted(sim_scores.items(), key=lambda item: item[1], reverse=True)[:5]


    return messages,target,sim_scores,top_5_statements

def generate_message_target_pair_with_top_5_staements(idx, train_contract_statements_and_annotations, nda, model=None, tokeniser=None):
    sim_scores = {}
    example_data = train_contract_statements_and_annotations[idx]
    hypothesis_embed = get_embedding(model, tokeniser, hypothesis_set[nda]['hypothesis'], 'mean')

    if model is not None and tokeniser is not None:
        for idx, statement in enumerate(example_data['statement'].split('\n')):
            if len(statement) > 0:
                statement_embed = get_embedding(model, tokeniser, statement, 'mean')
                # Store similarity scores
                statement_id = statement.split(']]')[0].split('[[')[-1]
                sim_scores[int(statement_id.split('statement_id_')[-1])] = (statement_embed @ hypothesis_embed.T).item()

    current_user_prompt = hypothesis_set[nda]['hypothesis']
    target = example_data['target']['annotations'][nda]

    # Get the top 5 items sorted by value
    top_5_statements = sorted(sim_scores.items(), key=lambda item: item[1], reverse=True)[:5]

    # Initialize system prompt components
    formatted_statements = []
    ids_to_format = set()  # To store unique IDs for formatting

    for statement_id, _ in top_5_statements:
        # Assume that statement_id is an integer or can be converted to an integer
        statement_id = int(statement_id)
        ids_to_format.update({statement_id-1, statement_id, statement_id + 1})
    ids_to_format = list(map(lambda x: f'statement_id_{x}', ids_to_format))
    print(ids_to_format)

    # Format statements only for the collected IDs
    for statement in example_data['statement'].split('\n'):
        # Assuming statement has a format where the ID is part of it
        statement_id =  statement.split(']]')[0].split('[[')[-1]  # Extract the ID
        if statement_id in ids_to_format:
            formatted_statements.append(statement)

    # Construct the system prompt using the filtered statements
    current_system_prompt = system_prompt.format('\n'.join(formatted_statements))

    # Construct the chat messages
    messages = [
        {"role": "system", "content": current_system_prompt},
        {"role": "user", "content": current_user_prompt},
    ]

    return messages, target, sim_scores, top_5_statements

In [ ]:
train_dataset = load_dataset(TRAIN_DATASET_PATH)
hypothesis_set = train_dataset['labels']
dev_dataset = load_dataset(DEV_DATASET_PATH)
tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True) 
test_dataset = load_dataset(TEST_DATASET_PATH)

train_contract_statements_and_annotations = construct_statement_annotation_pair(train_dataset['documents'])
dev_contract_statements_and_annotations = construct_statement_annotation_pair(dev_dataset['documents'])
test_contract_statements_and_annotations = construct_statement_annotation_pair(test_dataset['documents'])

In [4]:
max_seq = 7168
max_tok_string = f'{max_seq/1000:.0f}k'
# print(max_tok_string)
# train_dataset_processed, total_points, skipped_points = prepare_dataset_for_finetuning_method1(train_contract_statements_and_annotations,hypothesis_set,tokenizer,max_tokens=7168)
# train_dataset_processed = train_dataset_processed.map(lambda x: {'text': x['prompt'] + " " + x['completion']})
# train_dataset_processed.save_to_disk(f'{TRAIN_DATASET_PATH}_{max_tok_string}.hf_with_thoughts_and_opinion')
# print(total_points, skipped_points)
# dev_dataset_processed, total_points, skipped_points = prepare_dataset_for_finetuning_method1(dev_contract_statements_and_annotations,hypothesis_set,tokenizer,max_tokens=7168)
# dev_dataset_processed = dev_dataset_processed.map(lambda x: {'text': x['prompt'] + " " + x['completion']})
# dev_dataset_processed.save_to_disk(f'{DEV_DATASET_PATH}_{max_tok_string}.hf_with_thoughts_and_opinion')
# print(total_points, skipped_points)
test_dataset_processed, total_points, skipped_points = prepare_dataset_for_finetuning_method1(test_contract_statements_and_annotations,hypothesis_set,tokenizer,max_tokens=7168,augment_reason=False)
test_dataset_processed = test_dataset_processed.map(lambda x: {'text': x['prompt'] + " " + x['completion']})
test_dataset_processed.save_to_disk(f'{TEST_DATASET_PATH}_{max_tok_string}.hf_with_thoughts_and_opinion')
print(total_points, skipped_points)

NameError: name 'test_contract_statements_and_annotations' is not defined